# Food Delivery


To make up for his poor salary at THE EVIL COMPANY, the system admin works for a food delivery service on the way home. His job is to pick up meals from the five restaurants (r1,...,r5) in any order and bring them to the corresponding customers (c1,...,c5). After picking up one meal, he does not have to drive to the customer immediately. He can also pick up multiple meals first and then deliver them. The only constraint is that, on his bike, he can only carry three meals max. Travel times:

        work r1  r2  r3  r4  r5  c1  c2  c3  c4  c5 home
    work  -   6   3  22  20  10   -   -   -   -   -   -
      r1  -   -   5  28  18   4  22  17   8  23  26   -
      r2  -   4   -  23  18   7  17  12   8  19  21   -
      r3  -  25  21   -  21  27   5  17  22   6   4   -
      r4  -  20  20  25   -  16  19  28  10  17  26   -
      r5  -   5   9  30  16   -  23  21   6  24  28   -
      c1  -   -  16   6  16  21   -  14  16   3   6  16
      c2  -  14   -  17  24  17  14   -  17  17  14  15
      c3  -   9  10   -  10   6  19  21   -  18  24   3
      c4  -  22  18   8   -  22   4  19  16   -  10  17
      c5  -  23  18   4  22   -   5  13  21   8   -  20
    home  -   -   -   -   -   -   -   -   -   -   -   -
    
Labels on the left: starting points, labels on the top: destinations. For example, the trip from r3 to r4 takes 21 minutes. Useless connections are left empty. Download table as csv.

Find the shortest trip work - restaurant - ? - ? - ... - customer - home, such that each restaurant is visited before the corresponding customer and that the system admin never has more than three meals on his bike.

The solution code consists of the restaurants and customers in order (for example r1c1r2r3r4c2c4r5c5c3)

In [2]:
from itertools import permutations
import numpy as np
from numpy import genfromtxt

# Extract table from csv file
dist = genfromtxt('3_2_delivery_service.csv', delimiter=',')

# Generate permutations of all destinations, and use only the first half (route has to start either from 1-5,
# not allowed to start from 6-10)
nums = np.arange(1,11)
perm = list(permutations(nums))
perm = perm[:len(perm)//2]

# Store the restaurant and customer names in a dictionary, relating it to the integer values
destination_dict = {1:"r1", 2:"r2", 3:"r3", 4:"r4", 5:"r5",
                    6:"c1", 7:"c2", 8:"c3", 9:"c4", 10:"c5"}

perm_final = []

# Check the possible routes for maximum of 3 orders in the bag
for i in range(len(perm)):
    bag_capacity = []
    j = 0
    check = True
    while j < len(perm[i]) and check == True:
        
        # If the destination is a restaurant and there are less than 3 orders in the bag, allow the permutation
        if perm[i][j] <= 5 and len(bag_capacity) < 3:
            bag_capacity.append(perm[i][j])
        
        # If the destination is a restaurant and there are already 3 orders in the bag, don't allow the permutation
        elif perm[i][j] <= 5 and len(bag_capacity) == 3:
            check = False
        
        # If the destination is a customer and the order is already picked up, allow the permutation
        elif perm[i][j] > 5 and perm[i][j]-5 in bag_capacity:
            bag_capacity.remove(perm[i][j]-5)
        
        # If the destination is a customer and the order isn't already picked up, don't allow the permutation
        elif perm[i][j] > 5 and perm[i][j]-5 not in bag_capacity:
            check = False
        j += 1
    
    if check == True:
        perm_final.append(perm[i])

route = np.array(perm_final)

# Combine the permutations with work at the beginning and home at the end
work = np.zeros((route.shape[0],1))
home = np.ones((route.shape[0],1))*11

route = np.concatenate((work,route,home),axis=1)
route = route.astype(int)

# Initialize an array to store the distance for each corresponding route
route_dist = np.zeros(route.shape[0])

# Calculate distance for each corresponding route
for i in range(len(route_dist)):
    j = 0
    while j < route.shape[1]-1:
        route_dist[i] += dist[route[i,j],route[i,j+1]]
        j += 1

# Find the shortest route
shortest = np.where(route_dist == min(route_dist))
shortest = route[shortest].squeeze()

shortest_route = ''

# Convert the route from numbers to strings
for i in range(1,len(shortest)-1):
    shortest_route += destination_dict[shortest[i]]
    
print(shortest_route)

r2c2r1r5r4c4r3c5c1c3


## Solution = r2c2r1r5r4c4r3c5c1c3